In [ ]:
# Predict the price of the Uber ride, Linear, Lasso Ridge Regression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
uber = pd.read_csv("./datasets/uber.csv")
uber.shape

In [ ]:
uber.info()

In [ ]:
uber.sample(5)

In [ ]:
uber.drop(columns=["Unnamed: 0", "pickup_datetime"], inplace = True)
uber.isnull().sum() * 100 / len(uber)

In [ ]:
uber.dropna(axis=0, inplace=True)
uber.isnull().sum()

In [ ]:
uber.shape

#### Extracting Weekday From Key Column

In [ ]:
uber["key"] = pd.to_datetime(uber["key"])

In [ ]:
uber["weekday"] = uber["key"].dt.weekday

#### Computing Manhattan Distance in km from the coordinates

In [ ]:
lat_to_km = 111  # km per degree of latitude
long_to_km = 85  # km per degree of longitude

uber['manhattan_distance_km'] = (uber['pickup_latitude'] - uber['dropoff_latitude']).abs() * lat_to_km + \
(uber['pickup_longitude'] - uber['dropoff_longitude']).abs() * long_to_km

In [ ]:
uber.sample(5)

### 2. Identify outliers

In [ ]:
uber.describe()

In [ ]:
nyc_latitude_range = (40.5, 41.0)
nyc_longitude_range = (-74.5, -73.5)

# Filter the DataFrame to include only rows with coordinates within NYC-specific ranges
uber = uber[(uber['pickup_latitude'].between(nyc_latitude_range[0], nyc_latitude_range[1])) &
                (uber['pickup_longitude'].between(nyc_longitude_range[0], nyc_longitude_range[1])) &
                (uber['dropoff_latitude'].between(nyc_latitude_range[0], nyc_latitude_range[1])) &
                (uber['dropoff_longitude'].between(nyc_longitude_range[0], nyc_longitude_range[1]))]


In [ ]:
uber.shape

In [ ]:
uber.describe()

In [ ]:
# Function to detect outliers using IQR method with logical conditions
def get_outliers(df, column_name, logical_conditions):
    # Apply logical conditions to filter the DataFrame
    filtered_df = df.query(logical_conditions)

    Q1 = filtered_df[column_name].quantile(0.25)
    Q3 = filtered_df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    # Define lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = filtered_df[(filtered_df[column_name] < lower_bound) | (filtered_df[column_name] > upper_bound)]

    return lower_bound, upper_bound, outliers

In [ ]:
def uber_boxplot(df):
  # Plotting boxplots for visual inspection
  plt.figure(figsize=(18, 5))

  # Boxplot for fare_amount
  plt.subplot(1, 3, 1)
  sns.boxplot(data=df, y='fare_amount')
  plt.title('Fare Amount Boxplot')
  plt.ylabel('Fare Amount')

  # Boxplot for manhattan_distance_km
  plt.subplot(1, 3, 2)
  sns.boxplot(data=df, y='manhattan_distance_km')
  plt.title('Manhattan Distance Boxplot')
  plt.ylabel('Manhattan Distance (km)')

  plt.tight_layout()
  plt.show()

In [ ]:
uber_boxplot(uber)

In [ ]:
# Detect outliers for fare_amount
fare_lower, fare_upper, fare_outliers = get_outliers(
    uber, 'fare_amount', 'fare_amount > 0'
)

# Detect outliers for manhattan_distance_km
distance_lower, distance_upper, distance_outliers = get_outliers(
    uber, 'manhattan_distance_km', 'manhattan_distance_km > 1'
)

# Detect outliers for passenger_count
passenger_lower, passenger_upper, passenger_outliers = get_outliers(
    uber, 'passenger_count', '(passenger_count > 0) & (passenger_count <= 6)'
)


print(f"Fare Amount Bounds: ({fare_lower}, {fare_upper}), Outliers Detected: {fare_outliers.shape[0]}")
print(f"Manhattan Distance Bounds: ({distance_lower}, {distance_upper}), Outliers Detected: {distance_outliers.shape[0]}")
print(f"Passenger Count Bounds: ({passenger_lower}, {passenger_upper}), Outliers Detected: {passenger_outliers.shape[0]}")

In [ ]:
uber = uber[
    ((uber['fare_amount'] > 0) & (uber['fare_amount'] < fare_upper)) &
    ((uber['manhattan_distance_km'] < distance_upper) & (uber['manhattan_distance_km'] > 1)) &
    ((uber['passenger_count'] > 0) & (uber['passenger_count'] < passenger_upper))
]

uber.shape

In [ ]:
uber.describe()

In [ ]:
uber_boxplot(uber)

**Observation:**

- Many outliers are removed and we now have a cleaned dataset

### 3. Check the correlation

In [ ]:
# Calculate the correlation matrix for the filtered DataFrame
correlation_matrix_filtered = uber.corr()

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix_filtered, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title('Correlation Matrix (Filtered Data)')
plt.show()


### 4. Implement linear regression and ridge, Lasso regression models

In [ ]:
X = uber[['manhattan_distance_km']]
y = uber['fare_amount']

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create a Linear Regression model
linear_model = LinearRegression()
ridge_model = Ridge(alpha=1.0)
lasso_model = Lasso(alpha=0.1)

linear_model.fit(X_train, y_train)
ridge_model.fit(X_train, y_train)
lasso_model.fit(X_train, y_train)

# Make predictions
y_pred_linear = linear_model.predict(X_test)
y_pred_ridge = ridge_model.predict(X_test)
y_pred_lasso = lasso_model.predict(X_test)

# Evaluate the model
linear_rmse = mean_squared_error(y_test, y_pred_linear, squared=False)
linear_r2 = r2_score(y_test, y_pred_linear)

print(f"Linear Regression RMSE: {linear_rmse:.2f}, R^2: {linear_r2:.2f}")

In [ ]:
def evaluate_model(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return r2, rmse

In [ ]:
r2_linear, rmse_linear = evaluate_model(y_test, y_pred_linear)
r2_ridge, rmse_ridge = evaluate_model(y_test, y_pred_ridge)
r2_lasso, rmse_lasso = evaluate_model(y_test, y_pred_lasso)


print(f'Linear Regression: R² = {r2_linear:.2f}, RMSE = {rmse_linear:.2f}')
print(f'Ridge Regression: R² = {r2_ridge:.2f}, RMSE = {rmse_ridge:.2f}')
print(f'Lasso Regression: R² = {r2_lasso:.2f}, RMSE = {rmse_lasso:.2f}')

In [ ]:
# diabetes data set from UCI and Pima Indians Diabetes dataset, Univariate Multivariate analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from scipy import stats
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, r2_score, confusion_matrix

In [ ]:
pima_df = pd.read_csv('./datasets/diabetes.csv')
pima_df.shape

In [ ]:
pima_df.info()

In [ ]:
pima_df.sample(5)

In [ ]:
pima_df.describe()

#### a. Univariate Analysis

In [ ]:
# List of columns for univariate analysis
columns = pima_df.columns
columns

In [ ]:
# Function to compute univariate statistics
def univariate_analysis(df, cols):
    results = {}
    for col in cols:
        mean = df[col].mean()
        median = df[col].median()
        mode = df[col].mode()[0]  # Mode can have multiple values, so take the first one
        variance = df[col].var()
        std_dev = df[col].std()
        skewness = df[col].skew()
        kurtosis = df[col].kurtosis()

        results[col] = {
            'Mean': mean,
            'Median': median,
            'Mode': mode,
            'Variance': variance,
            'Standard Deviation': std_dev,
            'Skewness': skewness,
            'Kurtosis': kurtosis
        }

    return pd.DataFrame(results).T

In [ ]:
univariate_results = univariate_analysis(pima_df, columns)
univariate_results

In [ ]:
pima_df.sample(5)

#### b. Bivariate Analysis

In [ ]:
# Bivariate Analysis - Linear Regression
def linear_regression(df):
    print("\nBivariate Analysis - Linear Regression:")
    
    # Define independent and dependent variables
    X = df[['Glucose', 'BMI', 'Age']]  # Example predictors
    y = df['Outcome']

    # Add constant to predictor variables
    X = sm.add_constant(X)

    # Fit linear regression model
    model = sm.OLS(y, X).fit()
    
    # Print model summary
    print(model.summary())

# Bivariate Analysis - Logistic Regression
def logistic_regression(df):
    print("\nBivariate Analysis - Logistic Regression:")
    
    # Define independent and dependent variables
    X = df[['Glucose', 'BMI', 'Age']]
    y = df['Outcome']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit logistic regression model
    log_model = LogisticRegression(max_iter=200)
    log_model.fit(X_train, y_train)

    # Predict probabilities on the test set
    predictions = log_model.predict_proba(X_test)[:, 1]
    
    # Print coefficients and accuracy
    print("Logistic Regression Coefficients:\n", log_model.coef_)

In [ ]:
linear_regression(pima_df)

In [ ]:
logistic_regression(pima_df)

#### c. Multiple Regression

In [ ]:
# Multiple Regression Analysis
def multiple_regression(df):
    print("\nMultiple Regression Analysis:")
    
    X = df.drop(columns=['Outcome'])
    y = df['Outcome']
    
    # Add constant to predictor variables
    X = sm.add_constant(X)

    # Fit multiple regression model
    multi_model = sm.OLS(y, X).fit()
    
    # Print model summary
    print(multi_model.summary())

multiple_regression(pima_df)